In [ ]:
import os
import struct
import gc
import numpy as np
from keras.utils import np_utils
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
%matplotlib inline

# 1 . Create Dataset -- add random noise pics into MNIST

In [ ]:
from keras.datasets import cifar100
(X_train, y_train), (X_test, y_test) = cifar100.load_data() 
X_train = X_train / 255
X_test  = X_test / 255

In [ ]:
def gen_add_noise_50(X_train,y_train):
    
    noise_random = np.random.rand(50000,32,32,3)
    noise_labels = np.full((50000,1), 100, dtype=int)
    X_train = np.concatenate((X_train, noise_random), axis = 0 )
    y_train = np.concatenate((y_train, noise_labels), axis = 0 )
    return X_train,y_train

In [ ]:
def gen_add_noise_5(X_train,y_train):
    
    noise_random = np.random.rand(500,32,32,3)
    noise_labels = np.full((500,1), 100, dtype=int)
    X_train = np.concatenate((X_train, noise_random), axis = 0 )
    y_train = np.concatenate((y_train, noise_labels), axis = 0 )
    
    return X_train,y_train

In [ ]:
def shuffle_me(X_train,y_train):
    X_sparse_train = coo_matrix(X_train)
    X_train, X_sparse_train, y_train = shuffle(X_train, X_sparse_train, y_train, random_state=42)
    return X_train,y_train

In [ ]:
def keras_style_ds(X_train, X_test, y_train, y_test,original):
# keras - input
    if(original == 1):
        num_classes = 100               
    else:
        num_classes = 101
        
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test,  num_classes)
    
    return X_train, X_test, y_train, y_test, num_classes

# 2 . Build Model -- keras model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta


In [ ]:
def model_factory():
    
    K.clear_session()
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', 
                     data_format='channels_last',
                     input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
    
    return model


In [ ]:
# model = model_factory()
# model.summary()

# 3. Training

In [ ]:
# Train no random noise added
hs_mnist_no_scores = []
hs_mnist_no_history= []
for i in range(30):
    X_train_no,y_train_no = X_train,y_train
    X_train_no,y_train_no = shuffle_me(X_train_no,y_train_no)
    X_train_no, X_test_no, y_train_no, y_test_no, num_classes = keras_style_ds(X_train_no, X_test, 
                                                                               y_train_no, y_test, 1)
#     print(num_classes)
#     print(y_train_no.shape)
#     print(X_train_no.shape)
    model_no = model_factory()
#     model.summary()
    
    history_no = model_no.fit(X_train_no, y_train_no, epochs=12, batch_size=32,
                              validation_split=0.1,verbose=1)
    hs_mnist_no_history.append(history_no)
    
    model_no.save("./models/CIFAR100_dnn_0426_no_%d.h5" % (i+1))
    
    print()
    print("------------------------%d : test ------------------------------------------" % (i+1))
    test_no_score = model_no.evaluate(X_test_no, y_test_no, verbose=1)
    hs_mnist_no_scores.append(test_no_score)
    print("%d : CIFAR100 no noise test sets -> Loss: %.2f%%" % (i+1, test_no_score[0]))
    print("%d : CIFAR100 no noise test sets -> Accuracy: %.2f%%" % (i+1, test_no_score[1]*100))
    print()
    gc.collect()

In [ ]:
hs_mnist_no_scores

In [ ]:
# Train with 6000 random noise added
hs_mnist_6_scores = []
hs_mnist_6_history= []
for j in range(30):
    X_train_6,y_train_6 = gen_add_noise_5(X_train,y_train)
    X_train_6,y_train_6 = shuffle_me(X_train_6,y_train_6)
    X_train_6, X_test_6, y_train_6, y_test_6 , num_classes = keras_style_ds(X_train_6, X_test, 
                                                                            y_train_6, y_test, 0)
    
#     print(X_train_6.shape)
#     print(y_train_6.shape)
    model_6 = model_factory()
#     model_6.summary()
    
    history_6 = model_6.fit(X_train_6, y_train_6, epochs=12, batch_size = 32, 
                            validation_split = 0.1, verbose=1)
    hs_mnist_6_history.append(history_6)
    
    model_6.save("./models/CIFAR100_dnn_0426_5_%d.h5" % (j+1))
    
    print("------------------------%d : test ------------------------------------------" % (j+1))
    test_score_6 = model_6.evaluate(X_test_6, y_test_6, verbose=1)
    hs_mnist_6_scores.append(test_score_6)
    print("%d : CIFAR100 with 5 test sets -> Loss: %.2f%%" % (j+1, test_score_6[0]))
    print("%d : CIFAR100 with 5 test sets -> Accuracy: %.2f%%" % (j+1, test_score_6[1]*100))
    print()
    gc.collect()

In [ ]:
hs_mnist_6_scores

In [ ]:
# Train with 60,000 random noise added
hs_mnist_60_scores = []
hs_mnist_60_history= []
for h in range(30):
    
    X_train_60,y_train_60 = gen_add_noise_50(X_train,y_train)
    X_train_60,y_train_60 = shuffle_me(X_train_60,y_train_60)
    X_train_60, X_test_60, y_train_60, y_test_60, num_classes = keras_style_ds(X_train_60, X_test, 
                                                                               y_train_60, y_test, 0)
    
#     print(X_train_60.shape)
#     print(y_train_60.shape)
    model_60 = model_factory()
#     model_60.summary()
    
    history_60 = model_60.fit(X_train_60, y_train_60, epochs=12, batch_size=32,
                              validation_split=0.1,verbose=1)
    hs_mnist_60_history.append(history_60)
    
    model_60.save("./models/CIFAR100_dnn_0426_50_%d.h5" % (h+1))
    
    print("------------------------%d : test ------------------------------------------" % (h+1))
    test_score_60 = model_60.evaluate(X_test_60, y_test_60, verbose=1)
    hs_mnist_60_scores.append(test_score_60)
    print("%d : CIFAR100 with 50 test sets -> Loss: %.2f%%" % (h+1, test_score_60[0]))
    print("%d : CIFAR100 with 50 test sets -> Accuracy: %.2f%%" % (h+1, test_score_60[1]*100))
    print()
    gc.collect()

In [ ]:
hs_mnist_60_scores